In [1]:
# Monting Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Deusto/Cuarto/NLP/NLP Grupo/Codigo"

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1tbwtJBZplHfm91lceTlpEA9t4RR96dyM/NLP Grupo/Codigo


In [2]:
!pip install torchinfo
!mkdir model_storage

In [3]:
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from torchinfo import summary

# 🔤 Vocabuary Classes

In [ ]:
# @title Vocabulary

class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary

        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index

        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [ ]:
# @title SequenceVocabulary

class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token
          or the UNK index if token isn't present.

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
              for the UNK functionality
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

# 🔠 QuoteVectorizer & QuoteDataset

In [ ]:
# @title QuoteVectorizer

class QuoteVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, word_vocab, personajes_vocab):
        """
        Args:
            word_vocab (Vocabulary): maps words to integers
            personajes_vocab (Vocabulary): maps characters (personajes) to integers
        """
        self.word_vocab = word_vocab
        self.personajes_vocab = personajes_vocab

    def vectorize(self, quote, vector_length=-1):
        """Vectorize a quote into a vector of observations and targets

        The outputs are the vectorized quote split into two vectors:
            quote[:-1] and quote[1:]
        At each timestep, the first vector is the observation and the second vector is the target.

        Args:
            quote (str): the quote to be vectorized
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            a tuple: (from_vector, to_vector)
            from_vector (numpy.ndarray): the observation vector
            to_vector (numpy.ndarray): the target prediction vector
        """
        indices = [self.word_vocab.begin_seq_index]
        quote = quote.split(" ") # As tokens are now words
        indices.extend(self.word_vocab.lookup_token(token) for token in quote)
        indices.append(self.word_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices) - 1
        # ¿Qué hace esto?
        from_vector = np.empty(vector_length, dtype=np.int64)
        from_indices = indices[:-1]
        from_vector[:len(from_indices)] = from_indices
        from_vector[len(from_indices):] = self.word_vocab.mask_index

        #¿Y esto?
        to_vector = np.empty(vector_length, dtype=np.int64)
        to_indices = indices[1:]
        to_vector[:len(to_indices)] = to_indices
        to_vector[len(to_indices):] = self.word_vocab.mask_index

        return from_vector, to_vector

    @classmethod
    def from_dataframe(cls, quote_df):
        """Instantiate the vectorizer from the dataset dataframe

        Args:
            quote_df (pandas.DataFrame): the quote dataset
        Returns:
            an instance of the quoteVectorizer
        """
        word_vocab = SequenceVocabulary()
        personajes_vocab = Vocabulary()

        # Make changes here according to our dataframe
        for index, row in quote_df.iterrows():
            for word in row.quote_p.split(" "):
                word_vocab.add_token(word)
            personajes_vocab.add_token(row.char)

        return cls(word_vocab, personajes_vocab)

    @classmethod
    def from_serializable(cls, contents):
        """Instantiate the vectorizer from saved contents

        Args:
            contents (dict): a dict holding two vocabularies for this vectorizer
                This dictionary is created using `vectorizer.to_serializable()`
        Returns:
            an instance of quoteVectorizer
        """
        word_vocab = SequenceVocabulary.from_serializable(contents['word_vocab'])
        pj_vocab =  Vocabulary.from_serializable(contents['personajes_vocab'])

        return cls(word_vocab=word_vocab, personajes_vocab=pj_vocab)

    def to_serializable(self):
        """ Returns the serializable contents """
        return {'word_vocab': self.word_vocab.to_serializable(),
                'personajes_vocab': self.personajes_vocab.to_serializable()}

In [ ]:
# @title QuoteDataset

class QuoteDataset(Dataset):
    def __init__(self, quote_df, vectorizer):
        """
        Args:
            quote_df (pandas.DataFrame): the dataset
            vectorizer (QuoteVectorizer): vectorizer instatiated from dataset
        """
        self.quote_df = quote_df
        self._vectorizer = vectorizer

        self._max_seq_length = self.quote_df.length_quote.max() + 2 # It's the number of words on quote_p

        self.train_df = self.quote_df[self.quote_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.quote_df[self.quote_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.quote_df[self.quote_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, quote_csv):
        """Load dataset and make a new vectorizer from scratch

        Args:
            quote_csv (str): location of the dataset
        Returns:
            an instance of QuoteDataset
        """

        quote_df = pd.read_csv(quote_csv)
        return cls(quote_df, QuoteVectorizer.from_dataframe(quote_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, quote_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer.
        Used in the case in the vectorizer has been cached for re-use

        Args:
            quote_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of QuoteDataset
        """
        quote_df = pd.read_csv(quote_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(quote_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file

        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of QuoteVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return QuoteVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json

        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets

        Args:
            index (int): the index to the data point
        Returns:
            a dictionary holding the data point: (x_data, y_target, class_index)
        """
        row = self._target_df.iloc[index]

        from_vector, to_vector = \
            self._vectorizer.vectorize(row.quote_p, self._max_seq_length)

        personaje_index = \
            self._vectorizer.personajes_vocab.lookup_token(row.char) #Modified Revisar !!!!!!!

        return {'x_data': from_vector,
                'y_target': to_vector,
                'class_index': personaje_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset

        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

# 🔧 Other Utils

In [ ]:
# @title Other Utils

def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.
    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state


def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

# 💡Quote Generation Model

In [ ]:
# @title Model

class QuoteGenerationModel(nn.Module):
    def __init__(self, word_embedding_size, word_vocab_size, num_personajes, rnn_hidden_size,
                 batch_first=True, padding_idx=0, dropout_p=0.5):
        """
        Args:
            word_embedding_size (int): The size of the word embeddings
            word_vocab_size (int): The number of words to embed
            num_personajes (int): The size of the prediction vector
            rnn_hidden_size (int): The size of the RNN's hidden state
            batch_first (bool): Informs whether the input tensors will
                have batch or the sequence on the 0th dimension
            padding_idx (int): The index for the tensor padding;
                see torch.nn.Embedding
            dropout_p (float): the probability of zeroing activations using
                the dropout method.  higher means more likely to zero.
        """
        super(QuoteGenerationModel, self).__init__()

        self.word_emb = nn.Embedding(num_embeddings=word_vocab_size,
                                     embedding_dim=word_embedding_size,
                                     padding_idx=padding_idx)

        self.personaje_emb = nn.Embedding(num_embeddings=num_personajes,
                                       embedding_dim=rnn_hidden_size) #Modified

        self.rnn = nn.GRU(input_size=word_embedding_size,
                          hidden_size=rnn_hidden_size,
                          batch_first=batch_first)

        self.fc = nn.Linear(in_features=rnn_hidden_size,
                            out_features=word_vocab_size)

        self._dropout_p = dropout_p

    def forward(self, x_in, personaje_index, apply_softmax=False):
        """The forward pass of the model

        Args:
            x_in (torch.Tensor): an input data tensor.
                x_in.shape should be (batch, input_dim)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, word_vocab_size)
        """
        x_embedded = self.word_emb(x_in)

        personaje_embedded = self.personaje_emb(personaje_index).unsqueeze(0) #Modified


        y_out, _ = self.rnn(x_embedded, personaje_embedded) #Modified --> https://pytorch.org/docs/stable/generated/torch.nn.GRU.html

        batch_size, seq_size, feat_size = y_out.shape
        y_out = y_out.contiguous().view(batch_size * seq_size, feat_size)

        y_out = self.fc(F.dropout(y_out, p=self._dropout_p))

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        new_feat_size = y_out.shape[-1]
        y_out = y_out.view(batch_size, seq_size, new_feat_size)

        return y_out

In [ ]:
# @title Eval Model

def normalize_sizes(y_pred, y_true):
    """Normalize tensor sizes

    Args:
        y_pred (torch.Tensor): the output of the model
            If a 3-dimensional tensor, reshapes to a matrix
        y_true (torch.Tensor): the target predictions
            If a matrix, reshapes to be a vector
    """
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2)) #128*19, 88
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1) #128*19
    return y_pred, y_true

def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1) #Transformamos de #128*19, 88 a 128*19

    correct_indices = torch.eq(y_pred_indices, y_true).float() #Contamos los valores iguales
    valid_indices = torch.ne(y_true, mask_index).float() #Descartamos los valores que son iguales por el maskeo.

    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

In [ ]:
# @title Generate Samples

def sample_from_model(model, vectorizer, personajes, sample_size=20,
                      temperature=1.0):
    """Sample a sequence of indices from the model

    Args:
        model (QuoteGenerationModel): the trained model
        vectorizer (QuoteVectorizer): the corresponding vectorizer
        personajes (list): a list of integers representing personajes
        sample_size (int): the max length of the samples
        temperature (float): accentuates or flattens
            the distribution.
            0.0 < temperature < 1.0 will make it peakier.
            temperature > 1.0 will make it more uniform
    Returns:
        indices (torch.Tensor): the matrix of indices;
        shape = (num_samples, sample_size)
    """
    num_samples = len(personajes) #Modified
    begin_seq_index = [vectorizer.word_vocab.begin_seq_index
                       for _ in range(num_samples)]
    begin_seq_index = torch.tensor(begin_seq_index,
                                   dtype=torch.int64).unsqueeze(dim=1)
    indices = [begin_seq_index]
    personajes_indices = torch.tensor(personajes, dtype=torch.int64).unsqueeze(dim=0) #Modified
    h_t = model.personaje_emb(personajes_indices) #Modified

    for time_step in range(sample_size):
        x_t = indices[time_step]
        x_emb_t = model.word_emb(x_t)
        rnn_out_t, h_t = model.rnn(x_emb_t, h_t) #Modified
        prediction_vector = model.fc(rnn_out_t.squeeze(dim=1))
        probability_vector = F.softmax(prediction_vector / temperature, dim=1)
        indices.append(torch.multinomial(probability_vector, num_samples=1))
    indices = torch.stack(indices).squeeze().permute(1, 0)
    return indices


def decode_samples(sampled_indices, vectorizer):
    """Transform indices into the string form of a quote

    Args:
        sampled_indices (torch.Tensor): the inidces from `sample_from_model`
        vectorizer (QuoteVectorizer): the corresponding vectorizer
    """
    decoded_quotes = []
    vocab = vectorizer.word_vocab

    for sample_index in range(sampled_indices.shape[0]):
        quote = ""
        for time_step in range(sampled_indices.shape[1]):
            sample_item = sampled_indices[sample_index, time_step].item()
            if sample_item == vocab.begin_seq_index:
                continue
            elif sample_item == vocab.end_seq_index:
                break
            else:
                quote += vocab.lookup_index(sample_item)
                quote += " " # For separating words
        decoded_quotes.append(quote)
    return decoded_quotes

# 🧐 Analising Dataset

In [7]:
quote_df = pd.read_csv("../Datasets/dataset_rnn_quotes/dataset_quotes.csv")

def split_quote(q):
    return q.split(" ")
quote_df["list_words"] = quote_df.quote_p.apply(split_quote)
quote_df["length_quote"] = quote_df.list_words.apply(len)
print(quote_df["length_quote"].max())
print("Length DF: " + str(len(quote_df)))
quote_df = quote_df.loc[quote_df['length_quote'] <= 500]
print("Length DF after filter: " + str(len(quote_df)))
len_train = len(quote_df.loc[quote_df['split'] == "train"])
len_val = len(quote_df.loc[quote_df['split'] =="val"])
len_test = len(quote_df.loc[quote_df['split'] =="test"])
print(f"Train:{len_train} Val:{len_val} Test:{len_test}")

quote_df.to_csv("../Datasets/dataset_rnn_quotes/dataset_quotes_filtered.csv")
quote_df.head(5)

1125
Length DF: 10977
Length DF after filter: 10975
Train:7682 Val:1646 Test:1647


,char_id,char,quote_p,split,list_words,length_quote
0,292,Tresting,that thousand years of working in fields would...,val,"[that, thousand, years, of, working, in, field...",18
1,292,Tresting,oh half dozen or so,train,"[oh, half, dozen, or, so]",5
2,292,Tresting,some to beatings some to exhaustion,train,"[some, to, beatings, some, to, exhaustion]",6
3,292,Tresting,when first inherited this land from my father ...,val,"[when, first, inherited, this, land, from, my,...",42
4,292,Tresting,would you care to stay for supper,train,"[would, you, care, to, stay, for, supper]",7


# 🚅 Training Loop

In [ ]:
args = Namespace(
    # Data and Path information
    quote_csv="../Datasets/dataset_rnn_quotes/dataset_quotes_filtered.csv", # Loaded From the Rewrite
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="../Datasets/dataset_rnn_quotes/",
    # Model hyper parameters
    word_embedding_size=32,
    rnn_hidden_size=32,
    # Training hyper parameters
    seed=1337,
    learning_rate=0.001,
    batch_size=128, # Tal vez menos 16????????????????????????????????????????????????????????????????????????????????????????????????????
    num_epochs=500,
    early_stopping_criteria=5,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))


# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")

print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

if args.reload_from_files:
    # training from a checkpoint
    dataset = QuoteDataset.load_dataset_and_load_vectorizer(args.quote_csv,
                                                              args.vectorizer_file)
else:
    # create dataset and vectorizer
    dataset = QuoteDataset.load_dataset_and_make_vectorizer(args.quote_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()
print(f"The number of words is {len(vectorizer.word_vocab)}")
print(vectorizer.word_vocab._token_to_idx)
print(f"The number of personajes is {len(vectorizer.personajes_vocab)}")
print(vectorizer.personajes_vocab._token_to_idx)

model = QuoteGenerationModel(word_embedding_size=args.word_embedding_size,
                               word_vocab_size=len(vectorizer.word_vocab),
                               num_personajes=len(vectorizer.personajes_vocab),
                               rnn_hidden_size=args.rnn_hidden_size,
                               padding_idx=vectorizer.word_vocab.mask_index)

Expanded filepaths: 
	../Datasets/dataset_rnn_quotes/vectorizer.json
	../Datasets/dataset_rnn_quotes/model.pth
Using CUDA: False
The number of words is 7813
{'<MASK>': 0, '<UNK>': 1, '<BEGIN>': 2, '<END>': 3, 'that': 4, 'thousand': 5, 'years': 6, 'of': 7, 'working': 8, 'in': 9, 'fields': 10, 'would': 11, 'have': 12, 'bred': 13, 'them': 14, 'to': 15, 'be': 16, 'little': 17, 'more': 18, 'effective': 19, 'at': 20, 'it': 21, 'oh': 22, 'half': 23, 'dozen': 24, 'or': 25, 'so': 26, 'some': 27, 'beatings': 28, 'exhaustion': 29, 'when': 30, 'first': 31, 'inherited': 32, 'this': 33, 'land': 34, 'from': 35, 'my': 36, 'father': 37, 'had': 38, 'few': 39, 'runaways': 40, 'but': 41, 'executed': 42, 'their': 43, 'families': 44, 'the': 45, 'rest': 46, 'quickly': 47, 'lost': 48, 'heart': 49, 've': 50, 'never': 51, 'understood': 52, 'men': 53, 'who': 54, 'trouble': 55, 'with': 56, 'skaa': 57, 'find': 58, 'creatures': 59, 'easy': 60, 'control': 61, 'if': 62, 'you': 63, 'show': 64, 'properly': 65, 'firm': 

In [ ]:
mask_index = vectorizer.word_vocab.mask_index

model = model.to(args.device)


optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)
train_state = make_train_state(args)

epoch_bar = tqdm_notebook(desc='training routine',
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm_notebook(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size),
                          position=1,
                          leave=True)
dataset.set_split('val')
val_bar = tqdm_notebook(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size),
                        position=1,
                        leave=True)
summary_shown = False

do_train = False
if do_train:
  try:
      for epoch_index in range(args.num_epochs):
          train_state['epoch_index'] = epoch_index

          # Iterate over training dataset

          # setup: batch generator, set loss and acc to 0, set train mode on
          dataset.set_split('train')
          batch_generator = generate_batches(dataset,
                                            batch_size=args.batch_size,
                                            device=args.device)
          running_loss = 0.0
          running_acc = 0.0
          model.train()

          for batch_index, batch_dict in enumerate(batch_generator):
              if not summary_shown:
                  print(summary(model, input_data=[batch_dict['x_data'],batch_dict['class_index'] ]))
                  summary_shown = True
              # the training routine is these 5 steps:

              # --------------------------------------
              # step 1. zero the gradients
              optimizer.zero_grad()

              # step 2. compute the output
              y_pred = model(x_in=batch_dict['x_data'],
                            personaje_index=batch_dict['class_index']) #Modified

              # step 3. compute the loss
              loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)


              # step 4. use loss to produce gradients
              loss.backward()

              # step 5. use optimizer to take gradient step
              optimizer.step()
              # -----------------------------------------
              # compute the  running loss and running accuracy
              running_loss += (loss.item() - running_loss) / (batch_index + 1)
              acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
              running_acc += (acc_t - running_acc) / (batch_index + 1)

              # update bar
              train_bar.set_postfix(loss=running_loss,
                                    acc=running_acc,
                                    epoch=epoch_index)
              train_bar.update()

          train_state['train_loss'].append(running_loss)
          train_state['train_acc'].append(running_acc)

          # Iterate over val dataset

          # setup: batch generator, set loss and acc to 0; set eval mode on
          dataset.set_split('val')
          batch_generator = generate_batches(dataset,
                                            batch_size=args.batch_size,
                                            device=args.device)
          running_loss = 0.
          running_acc = 0.
          model.eval()

          for batch_index, batch_dict in enumerate(batch_generator):
              # compute the output
              y_pred = model(x_in=batch_dict['x_data'], personaje_index=batch_dict['class_index']) #Modified

              # step 3. compute the loss
              loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)

              # compute the  running loss and running accuracy
              running_loss += (loss.item() - running_loss) / (batch_index + 1)
              acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
              running_acc += (acc_t - running_acc) / (batch_index + 1)

              # Update bar
              val_bar.set_postfix(loss=running_loss, acc=running_acc,
                              epoch=epoch_index)
              val_bar.update()

          train_state['val_loss'].append(running_loss)
          train_state['val_acc'].append(running_acc)

          train_state = update_train_state(args=args, model=model,
                                          train_state=train_state)

          scheduler.step(train_state['val_loss'][-1])

          if train_state['stop_early']:
              break

          train_bar.n = 0
          val_bar.n = 0
          epoch_bar.update()

  except KeyboardInterrupt:
      print("Exiting loop")

<ipython-input-14-249b95dafb75>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  epoch_bar = tqdm_notebook(desc='training routine',


training routine:   0%|          | 0/500 [00:00<?, ?it/s]

<ipython-input-14-249b95dafb75>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  train_bar = tqdm_notebook(desc='split=train',


split=train:   0%|          | 0/60 [00:00<?, ?it/s]

<ipython-input-14-249b95dafb75>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  val_bar = tqdm_notebook(desc='split=val',


split=val:   0%|          | 0/12 [00:00<?, ?it/s]

# 🌟 Final Testing

In [ ]:
# compute the loss & accuracy on the test set using the best available model

do_test = False
if do_test:
  model.load_state_dict(torch.load(train_state['model_filename']))

  model = model.to(args.device)

  dataset.set_split('test')
  batch_generator = generate_batches(dataset,
                                    batch_size=args.batch_size,
                                    device=args.device)
  running_loss = 0.
  running_acc = 0.
  model.eval()

  for batch_index, batch_dict in enumerate(batch_generator):
      # compute the output
      y_pred = model(x_in=batch_dict['x_data'],  personaje_index=batch_dict['class_index']) #Modified

      # compute the loss
      loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)

      # compute the accuracy
      running_loss += (loss.item() - running_loss) / (batch_index + 1)

      acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
      running_acc += (acc_t - running_acc) / (batch_index + 1)

  train_state['test_loss'] = running_loss
  train_state['test_acc'] = running_acc

  print("Test loss: {};".format(train_state['test_loss']))
  print("Test Accuracy: {}".format(train_state['test_acc']))

In [ ]:
model = model.cpu()
for index in range(len(vectorizer.personajes_vocab)):
    personajes = vectorizer.personajes_vocab.lookup_index(index)
    print("Sampled for {}: ".format(personajes)) #Modified
    sampled_indices = sample_from_model(model, vectorizer,
                                        personajes=[index] * 5, # Number of quotes per character
                                        sample_size= 500, # Max length
                                        temperature=1.0) # Distribution (1 = Normal)
    for sampled_quote in decode_samples(sampled_indices, vectorizer):
        print("-  " + sampled_quote)

Output hidden; open in https://colab.research.google.com to view.